<a href="https://colab.research.google.com/github/julianovale/MCDA/blob/main/DEA_Data_Envelopment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DEA - Data Envelopment Analysis**

Implementação disponível em: https://github.com/metjush/envelopment-py

In [ ]:
"""
Data Envelopment Analysis implementation
Sources:
Sherman & Zhu (2006) Service Productivity Management, Improving Service Performance using Data Envelopment Analysis (DEA) [Chapter 2]
ISBN: 978-0-387-33211-6
http://deazone.com/en/resources/tutorial
"""

# Execute esta célula para compilar a função dea(X,Y)
# Modelo CCR orientado a inputs

import numpy as np
from scipy.optimize import fmin_slsqp


class DEA(object):

    def __init__(self, inputs, outputs):
        """
        Initialize the DEA object with input data
        n = number of entities (observations)
        m = number of inputs (variables, features)
        r = number of outputs
        :param inputs: inputs, n x m numpy array
        :param outputs: outputs, n x r numpy array
        :return: self
        """

        # supplied data
        self.inputs = inputs
        self.outputs = outputs

        # parameters
        self.n = inputs.shape[0]
        self.m = inputs.shape[1]
        self.r = outputs.shape[1]

        # iterators
        self.unit_ = range(self.n)
        self.input_ = range(self.m)
        self.output_ = range(self.r)

        # result arrays
        self.output_w = np.zeros((self.r, 1), dtype=np.float)  # output weights
        self.input_w = np.zeros((self.m, 1), dtype=np.float)  # input weights
        self.lambdas = np.zeros((self.n, 1), dtype=np.float)  # unit efficiencies
        self.efficiency = np.zeros_like(self.lambdas)  # thetas

        # names
        self.names = []

    def __efficiency(self, unit):
        """
        Efficiency function with already computed weights
        :param unit: which unit to compute for
        :return: efficiency
        """

        # compute efficiency
        denominator = np.dot(self.inputs, self.input_w)
        numerator = np.dot(self.outputs, self.output_w)

        return (numerator/denominator)[unit]

    def __target(self, x, unit):
        """
        Theta target function for one unit
        :param x: combined weights
        :param unit: which production unit to compute
        :return: theta
        """
        in_w, out_w, lambdas = x[:self.m], x[self.m:(self.m+self.r)], x[(self.m+self.r):]  # unroll the weights
        denominator = np.dot(self.inputs[unit], in_w)
        numerator = np.dot(self.outputs[unit], out_w)

        return numerator/denominator

    def __constraints(self, x, unit):
        """
        Constraints for optimization for one unit
        :param x: combined weights
        :param unit: which production unit to compute
        :return: array of constraints
        """

        in_w, out_w, lambdas = x[:self.m], x[self.m:(self.m+self.r)], x[(self.m+self.r):]  # unroll the weights
        constr = []  # init the constraint array

        # for each input, lambdas with inputs
        for input in self.input_:
            t = self.__target(x, unit)
            lhs = np.dot(self.inputs[:, input], lambdas)
            cons = t*self.inputs[unit, input] - lhs
            constr.append(cons)

        # for each output, lambdas with outputs
        for output in self.output_:
            lhs = np.dot(self.outputs[:, output], lambdas)
            cons = lhs - self.outputs[unit, output]
            constr.append(cons)

        # for each unit
        for u in self.unit_:
            constr.append(lambdas[u])

        return np.array(constr)

    def __optimize(self):
        """
        Optimization of the DEA model
        Use: http://docs.scipy.org/doc/scipy-0.17.0/reference/generated/scipy.optimize.linprog.html
        A = coefficients in the constraints
        b = rhs of constraints
        c = coefficients of the target function
        :return:
        """
        d0 = self.m + self.r + self.n
        # iterate over units
        for unit in self.unit_:
            # weights
            x0 = np.random.rand(d0) - 0.5
            x0 = fmin_slsqp(self.__target, x0, f_ieqcons=self.__constraints, args=(unit,))
            # unroll weights
            self.input_w, self.output_w, self.lambdas = x0[:self.m], x0[self.m:(self.m+self.r)], x0[(self.m+self.r):]
            self.efficiency[unit] = self.__efficiency(unit)

    def name_units(self, names):
        """
        Provide names for units for presentation purposes
        :param names: a list of names, equal in length to the number of units
        :return: nothing
        """

        assert(self.n == len(names))

        self.names = names

    def fit(self):
        """
        Optimize the dataset, generate basic table
        :return: table
        """

        self.__optimize()  # optimize

        print("Final thetas for each DMU:\n")
        print("---------------------------\n")
        for n, eff in enumerate(self.efficiency):
            if len(self.names) > 0:
                name = "DMU: %s" % self.names[n]
            else:
                name = "DMU: %d" % (n+1)
            print("%s theta: %.4f" % (name, eff))
            print("\n")
        print("---------------------------\n")

**Entrada de dados**

In [ ]:
# Entre com a matriz de Inputs
X = np.array([[14.2 , 32.44],
       [ 5.  ,  9.37],
       [14.3 ,  8.28],
       [12.9 , 38.64],
       [13.9 , 19.84],
       [ 5.  , 28.96],
       [ 8.5 , 25.09],
       [ 5.5 , 33.1 ],
       [10.  , 38.73],
       [13.5 , 36.67],
       [11.  , 14.17],
       [ 0.8 , 14.58]])

# Entre com a matriz de Inputs
Y = np.array(
      [[0.75,0.88,0.95], 
       [0.87,0.92,0.96],
       [0.77,0.94,0.97],
       [0.85,0.93,0.95],
       [0.99,0.91,0.99],
       [0.84,0.84,0.97],
       [0.69,0.88,0.95],
       [0.83,0.93,0.97],
       [0.84,0.82,0.95],
       [0.89,0.94,0.96],
       [0.58,0.85,0.99],
       [0.72,0.81,0.97]])

# Entre com os nomes das DMUs em forma de lista
names = ['Álvaro de Carvalho', 'Araraquara','Avaré','Baurú','Getulina','Iaras','Itaí', 'Lins', 'Marília', 'Pirajuí', 'Ribeirão Preto', 'Serra Azul']

In [ ]:
#Crie o problema dea através do comando:
dea = DEA(X,Y)
#Inclua o nome das DMUs através do comando:
dea.name_units(names)

In [ ]:
#Resolva os PLs e imprima os valores das eficiências:
dea.fit();

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.30742222173523726
            Iterations: 10
            Function evaluations: 191
            Gradient evaluations: 10
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.000000049444246
            Iterations: 14
            Function evaluations: 271
            Gradient evaluations: 14
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.0000002382249207
            Iterations: 13
            Function evaluations: 254
            Gradient evaluations: 13
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.29936679529923127
            Iterations: 11
            Function evaluations: 210
            Gradient evaluations: 11
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.5373021197877754
            Iterations: 14
            Fu

In [ ]:
# Entre com a matriz de Inputs
X = np.array([[5,14],
       [8,15],
       [7,12]])

# Entre com a matriz de Inputs
Y = np.array(
      [[9,4,16], 
       [5,7,10],
       [4,9,13]])

# Entre com os nomes das DMUs em forma de lista
names = ['H1', 'H2', 'H3']

In [ ]:
#Crie o problema dea através do comando:
dea = DEA(X,Y)
#Inclua o nome das DMUs através do comando:
dea.name_units(names)
#Resolva os PLs e imprima os valores das eficiências:
dea.fit();

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.0000000577979504
            Iterations: 3
            Function evaluations: 30
            Gradient evaluations: 3
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.7733333351264237
            Iterations: 7
            Function evaluations: 70
            Gradient evaluations: 7
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.0000000283169828
            Iterations: 6
            Function evaluations: 60
            Gradient evaluations: 6
Final thetas for each DMU:

---------------------------

DMU: H1 theta: 1.0000


DMU: H2 theta: 0.7733


DMU: H3 theta: 1.0000


---------------------------

